In [32]:
import pandas
import string
import nltk
import keras
import sklearn
import time
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import numpy as np
from time import time
from nltk.corpus import stopwords
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import keras.preprocessing.text

In [33]:
Train_X = pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['text']
Test_X = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['text']

Train_Y_category= pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['category']

Test_Y_category= pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['category']

In [34]:
# concatenate both dev and train texts
texts = Train_X.append(Test_X, ignore_index=True)


tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

Train_X = tokenizer.texts_to_matrix(Train_X, mode='tfidf')
Test_X = tokenizer.texts_to_matrix(Test_X, mode='tfidf')

C:\Users\hazem\AppData\Local\Temp\ipykernel_16200\1537059493.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts = Train_X.append(Test_X, ignore_index=True)


In [35]:
# concatenate the stance labels in dev and train
categories = Train_Y_category.append(Test_Y_category, ignore_index=True)

# encoding my labels from text to numbers not necessary for stance
encoder = LabelEncoder()
encoder.fit(categories)
categories=encoder.fit_transform(categories)

num_classes = int((len(set(categories))))
print((len(set(categories))))

# tranforming my labels to their encoded numbers
Train_Y_stance = encoder.fit_transform(Train_Y_category)
Test_Y_stance = encoder.fit_transform(Test_Y_category)

3


C:\Users\hazem\AppData\Local\Temp\ipykernel_16200\668322745.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stances = Train_Y_stance.append(Test_Y_stance, ignore_index=True)


In [36]:
Train_Y_category= keras.utils.to_categorical(Train_Y_category,num_classes)
Test_Y_category = keras.utils.to_categorical(Test_Y_category, num_classes)


num_labels = int(len(Train_Y_category.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size

In [37]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives

In [38]:
# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])

In [40]:
batch_size = 100
epochs = 4

history = model.fit(Train_X, Train_Y_stance,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


model.save('my_model.h1')

Epoch 1/4
63/63 [==============================] - 30s 467ms/step - loss: 0.5995 - categorical_accuracy: 0.7907 - recall: 0.8889 - precision: 0.6523 - f1_metric: 0.7546 - true_positives: 5590.0000 - true_negatives: 9598.0000 - false_positives: 2980.0000 - false_negatives: 699.0000 - val_loss: 0.5314 - val_categorical_accuracy: 0.7983 - val_recall: 0.9342 - val_precision: 0.6267 - val_f1_metric: 0.7506 - val_true_positives: 653.0000 - val_true_negatives: 1009.0000 - val_false_positives: 389.0000 - val_false_negatives: 46.0000
Epoch 2/4
63/63 [==============================] - 27s 431ms/step - loss: 0.1433 - categorical_accuracy: 0.9595 - recall: 0.9916 - precision: 0.7803 - f1_metric: 0.8739 - true_positives: 6236.0000 - true_negatives: 10822.0000 - false_positives: 1756.0000 - false_negatives: 53.0000 - val_loss: 0.6487 - val_categorical_accuracy: 0.8069 - val_recall: 0.9084 - val_precision: 0.6813 - val_f1_metric: 0.7792 - val_true_positives: 635.0000 - val_true_negatives: 1101.0000 -

INFO:tensorflow:Assets written to: my_model.h1\assets


INFO:tensorflow:Assets written to: my_model.h1\assets


In [41]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(Test_X,Test_Y_stance,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)

Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[0.8397632837295532, 0.7950000166893005, 0.8830000162124634, 0.6985759735107422, 0.7830392718315125, 883.0, 1619.0, 381.0, 117.0]


In [42]:
import csv

Test_X = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['text']
Test_Y_stance = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['stance']

In [43]:
fields = ['Text', 'Predicted', 'Actual','isCorrect']

filename = "../Results/AraBert.csv"

correct_count = 0
predictions=[]

with open(filename, 'w',encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)

    for i in range(len(Test_X)):
        tokens = tokenizer.texts_to_matrix([Test_X[i]], mode='tfidf')
        predictions_vector =model.predict(np.array(tokens))
        predicted_class_encoded =np.argmax(predictions_vector,axis=1)
        predicted_class_name = encoder.inverse_transform(predicted_class_encoded)[0]

        isCorrect = "False"
        predictions.append(predicted_class_name)
        if predicted_class_name == Test_Y_stance[i]:
            correct_count += 1
            isCorrect = "True"
        
        csvwriter.writerow([Test_X[i], str(predicted_class_name), str(Test_Y_stance[i]), isCorrect])

1/1 [==============================] - 0s 26ms/step


In [30]:
from sklearn.metrics import f1_score

accuracy = correct_count / len(Test_X)
print("F1 score ", f1_score(Test_Y_stance, predictions, average='macro'))
print("Accuracy: ", accuracy)


F1 score  0.47907814690061573
Accuracy:  0.798
